### Importing libraries

In [1]:
% pylab inline
import os
import pandas as pd
import librosa
import glob 
import librosa.display

Populating the interactive namespace from numpy and matplotlib


In [2]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

Using TensorFlow backend.


In [6]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 

### Importing the audio labels (rating of audio files from 1-5 in the advertisements)

In [9]:
#labels basepath
Basepath='C:/Users/asd/Desktop/Research Work/videoAds/'
List = open(Basepath+"labels1000.txt").readlines()
List2=[]#list 2 stores the labels of each audio present to us
for i in range(len(List)):
    List2.append(List[i].split("\n")[0])

In [11]:
Labels=np.asarray(List2)#labels are added as numpy array
Labels = Labels.astype(np.int)#integer array is made from the strings array
print(Labels.shape)
print(max(Labels))
print(min(Labels))
#print(Labels)

(800,)
5
1


### Conversion of labels in oneHot format

In [7]:
hotLabel = np.zeros((len(Labels), max(Labels)+1))
hotLabel[np.arange(len(Labels)),Labels] = 1#hotLabel is the oneHot format of the labels
print(hotLabel)

[[ 0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  1.]
 [ 0.  0.  1.  0.  0.  0.]
 ..., 
 [ 0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.]]


All the audios we have have been transformed to hold the numeric names for processing using the ExtractAudio code present in this repositry. 

The names of audios are `1.wav`, `2.wav` and so on

In [8]:
tit=glob.glob(Basepath+'Extract/*.wav')#tit represents the title list of all the audio files we have
#print(tit)

In [9]:
tit2=[]#tit2 is the titles stored in the list
for i in range(len(tit)):
    tit2.append(tit[i].split('\\')[1])

In [10]:
nptit2=np.asarray(tit2)#titles are stored as numpy array of strings
print(nptit2.shape)

(680,)


In [11]:
titNo=[]#title number are stored(alternative way) as a list
for i in range(len(tit)):
    titNo.append(tit2[i].split('.')[0])
#print(titNo)

In [12]:
nptitNo=np.asarray(titNo)
intTitNo=[]#integral title numbers are stored here
for i in range(len(nptitNo)):
    intTitNo.append(int(nptitNo[i]))
#print(intTitNo)

Other way to store the title numbers

In [13]:
nptitNo = [int(x) for x in nptitNo]
nptitNo.sort()
#print(nptitNo)

In [14]:
intTitNo=nptitNo
#print(intTitNo)

In [15]:
totalHotLabel=[]
for x in intTitNo:
    totalHotLabel.append(hotLabel[x])
totalHotLabel=np.asarray(totalHotLabel)
# print(totalHotLabel[0])
# print("hi\n",hotLabel[1])

In [16]:
titles=[]
for x in intTitNo:
    titles.append(str(x)+".wav")

### The parser function is used to extract the audios one by one in mfcc(Mel Frequency Cepstrums) format

This may take upto `5 minutes` for `10 audios` depending upon the laptop

In [54]:
# def parser(title):
#     # function to load files and extract features
#     #print(row)
#     file_name = os.path.join(Basepath, 'Extract',  title)
#     # handle exception to check if there isn't a file which is corrupted
#     try:
#         # here kaiser_fast is a technique used for faster extraction
#         X, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
#         # we extract mfcc feature from data
#         mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0) 
#     except Exception as e:
#         print("Error encountered while parsing file: ", file_name)
#         return None, None
#     feature = mfccs
# #     label = row.Class
#     return feature#, label]
# parsed=[]
# for i in range(len(titles)):
#     parsed.append(parser(titles[i]))
# #temp.columns = ['feature', 'label']

In [56]:
# print(parsed[0])
# print(totalHotLabel[0])

In [19]:
# parsed2=np.asarray(parsed) #Storing the parsed mfcc format audio files are stored as numpy arrays
# print(parsed2.shape)

### The mfcc format audio files are saved here as a pickle file and later, the file is used for extracting the audio informtion to save time

In [61]:
# import pickle
# with open('Audio1000.txt', 'wb') as file:
#      file.write(pickle.dumps(parsed))

In [18]:
import pickle
with open('Audio1000.txt', 'rb') as f:
    temp = pickle.load(f)
print(np.asarray(temp).shape)

(680, 40)


# Data Training and slicing into validation and training

In [20]:
X = np.array(temp[0:600])#Audio files in mfcc format sliced for training
y = np.array(totalHotLabel[0:600].tolist())#Labels sliced for training
Xvalid = np.array(temp[600:680])#validation audio
yvalid = np.array(totalHotLabel[600:680].tolist())#validation labels

In [22]:
print(X.shape)
print(y.shape)
print(Xvalid.shape)
print(yvalid.shape)

(600, 40)
(600, 6)
(80, 40)
(80, 6)


In [ ]:
model = Sequential()

model.add(Dense(20, input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(20))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(6))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

model.fit(X, y, batch_size=30, epochs=50, validation_data=(Xvalid, yvalid))

In [ ]:
# Train on 600 samples, validate on 80 samples
# Epoch 1/50
# 600/600 [==============================] - 1s 2ms/step - loss: 9.5760 - acc: 0.3333 - val_loss: 8.8740 - val_acc: 0.3750
# Epoch 2/50
# 600/600 [==============================] - 0s 208us/step - loss: 9.3599 - acc: 0.3533 - val_loss: 6.2563 - val_acc: 0.3750
# Epoch 3/50
# 600/600 [==============================] - 0s 208us/step - loss: 9.2070 - acc: 0.3583 - val_loss: 7.7954 - val_acc: 0.3500
# Epoch 4/50
# 600/600 [==============================] - 0s 156us/step - loss: 9.1745 - acc: 0.3717 - val_loss: 10.1998 - val_acc: 0.3625
# Epoch 5/50
# 600/600 [==============================] - 0s 156us/step - loss: 9.1768 - acc: 0.4150 - val_loss: 9.8792 - val_acc: 0.3625
# Epoch 6/50
# 600/600 [==============================] - 0s 156us/step - loss: 8.9767 - acc: 0.4000 - val_loss: 9.8124 - val_acc: 0.3875
# Epoch 7/50
# 600/600 [==============================] - 0s 182us/step - loss: 8.4750 - acc: 0.4367 - val_loss: 8.4497 - val_acc: 0.4000
# Epoch 8/50
# 600/600 [==============================] - 0s 182us/step - loss: 8.9892 - acc: 0.3750 - val_loss: 8.2213 - val_acc: 0.4000
# Epoch 9/50
# 600/600 [==============================] - 0s 156us/step - loss: 9.0653 - acc: 0.3783 - val_loss: 9.3038 - val_acc: 0.3875
# Epoch 10/50
# 600/600 [==============================] - 0s 182us/step - loss: 8.9319 - acc: 0.4000 - val_loss: 9.6328 - val_acc: 0.3750
# Epoch 11/50
# 600/600 [==============================] - 0s 156us/step - loss: 8.8559 - acc: 0.3833 - val_loss: 8.3971 - val_acc: 0.4375
# Epoch 12/50
# 600/600 [==============================] - 0s 182us/step - loss: 9.0266 - acc: 0.3850 - val_loss: 9.1209 - val_acc: 0.3875
# Epoch 13/50
# 600/600 [==============================] - 0s 156us/step - loss: 8.7546 - acc: 0.3983 - val_loss: 8.2704 - val_acc: 0.4000
# Epoch 14/50
# 600/600 [==============================] - 0s 182us/step - loss: 8.6737 - acc: 0.3950 - val_loss: 7.8393 - val_acc: 0.4250
# Epoch 15/50
# 600/600 [==============================] - 0s 182us/step - loss: 8.4143 - acc: 0.3933 - val_loss: 9.7829 - val_acc: 0.3875
# Epoch 16/50
# 600/600 [==============================] - 0s 156us/step - loss: 8.6687 - acc: 0.4017 - val_loss: 7.3882 - val_acc: 0.4000
# Epoch 17/50
# 600/600 [==============================] - 0s 182us/step - loss: 8.2197 - acc: 0.3967 - val_loss: 7.3774 - val_acc: 0.3875
# Epoch 18/50
# 600/600 [==============================] - 0s 182us/step - loss: 8.5341 - acc: 0.3850 - val_loss: 9.1661 - val_acc: 0.3750
# Epoch 19/50
# 600/600 [==============================] - 0s 182us/step - loss: 8.4214 - acc: 0.4100 - val_loss: 9.3875 - val_acc: 0.3875
# Epoch 20/50
# 600/600 [==============================] - 0s 156us/step - loss: 7.9397 - acc: 0.4067 - val_loss: 7.5088 - val_acc: 0.4125
# Epoch 21/50
# 600/600 [==============================] - 0s 182us/step - loss: 7.5130 - acc: 0.4167 - val_loss: 7.1436 - val_acc: 0.3750
# Epoch 22/50
# 600/600 [==============================] - 0s 182us/step - loss: 7.9302 - acc: 0.3767 - val_loss: 6.7042 - val_acc: 0.3625
# Epoch 23/50
# 600/600 [==============================] - 0s 182us/step - loss: 6.9926 - acc: 0.3750 - val_loss: 5.2409 - val_acc: 0.4125
# Epoch 24/50
# 600/600 [==============================] - 0s 182us/step - loss: 6.4289 - acc: 0.3750 - val_loss: 5.2665 - val_acc: 0.4000
# Epoch 25/50
# 600/600 [==============================] - 0s 182us/step - loss: 5.2470 - acc: 0.4050 - val_loss: 2.9997 - val_acc: 0.4250
# Epoch 26/50
# 600/600 [==============================] - 0s 182us/step - loss: 4.3575 - acc: 0.3133 - val_loss: 1.6546 - val_acc: 0.4000
# Epoch 27/50
# 600/600 [==============================] - 0s 208us/step - loss: 3.1019 - acc: 0.3000 - val_loss: 1.5328 - val_acc: 0.3875
# Epoch 28/50
# 600/600 [==============================] - 0s 339us/step - loss: 2.3796 - acc: 0.3300 - val_loss: 1.5869 - val_acc: 0.3375
# Epoch 29/50
# 600/600 [==============================] - 0s 234us/step - loss: 1.9815 - acc: 0.3400 - val_loss: 1.5891 - val_acc: 0.3750
# Epoch 30/50
# 600/600 [==============================] - 0s 339us/step - loss: 1.7521 - acc: 0.3400 - val_loss: 1.5959 - val_acc: 0.3250
# Epoch 31/50
# 600/600 [==============================] - 0s 260us/step - loss: 1.6638 - acc: 0.4050 - val_loss: 1.6127 - val_acc: 0.3125
# Epoch 32/50
# 600/600 [==============================] - 0s 312us/step - loss: 1.6730 - acc: 0.4050 - val_loss: 1.5836 - val_acc: 0.3375
# Epoch 33/50
# 600/600 [==============================] - 0s 260us/step - loss: 1.7081 - acc: 0.3617 - val_loss: 1.5636 - val_acc: 0.3625
# Epoch 34/50
# 600/600 [==============================] - 0s 208us/step - loss: 1.6002 - acc: 0.3900 - val_loss: 1.5442 - val_acc: 0.3625
# Epoch 35/50
# 600/600 [==============================] - 0s 286us/step - loss: 1.4700 - acc: 0.3650 - val_loss: 1.4980 - val_acc: 0.3375
# Epoch 36/50
# 600/600 [==============================] - 0s 182us/step - loss: 1.5017 - acc: 0.3883 - val_loss: 1.4879 - val_acc: 0.4000
# Epoch 37/50
# 600/600 [==============================] - 0s 156us/step - loss: 1.4762 - acc: 0.4150 - val_loss: 1.4869 - val_acc: 0.3875
# Epoch 38/50
# 600/600 [==============================] - 0s 208us/step - loss: 1.4553 - acc: 0.4033 - val_loss: 1.4732 - val_acc: 0.3625
# Epoch 39/50
# 600/600 [==============================] - 0s 130us/step - loss: 1.4314 - acc: 0.4083 - val_loss: 1.4674 - val_acc: 0.3625
# Epoch 40/50
# 600/600 [==============================] - 0s 156us/step - loss: 1.4280 - acc: 0.4083 - val_loss: 1.4594 - val_acc: 0.3875
# Epoch 41/50
# 600/600 [==============================] - 0s 156us/step - loss: 1.4319 - acc: 0.3950 - val_loss: 1.4565 - val_acc: 0.3875
# Epoch 42/50
# 600/600 [==============================] - 0s 182us/step - loss: 1.4195 - acc: 0.4367 - val_loss: 1.4551 - val_acc: 0.4000
# Epoch 43/50
# 600/600 [==============================] - 0s 156us/step - loss: 1.4240 - acc: 0.4167 - val_loss: 1.4615 - val_acc: 0.3875c
# Epoch 44/50
# 600/600 [==============================] - 0s 156us/step - loss: 1.3902 - acc: 0.4317 - val_loss: 1.4554 - val_acc: 0.3750
# Epoch 45/50
# 600/600 [==============================] - 0s 156us/step - loss: 1.3972 - acc: 0.4217 - val_loss: 1.4528 - val_acc: 0.3750
# Epoch 46/50
# 600/600 [==============================] - 0s 156us/step - loss: 1.4279 - acc: 0.4100 - val_loss: 1.4492 - val_acc: 0.3500
# Epoch 47/50
# 600/600 [==============================] - 0s 260us/step - loss: 1.4258 - acc: 0.3767 - val_loss: 1.4515 - val_acc: 0.3625
# Epoch 48/50
# 600/600 [==============================] - 0s 313us/step - loss: 1.3783 - acc: 0.4417 - val_loss: 1.4471 - val_acc: 0.3625
# Epoch 49/50
# 600/600 [==============================] - 0s 365us/step - loss: 1.4088 - acc: 0.4100 - val_loss: 1.4506 - val_acc: 0.3625
# Epoch 50/50
# 600/600 [==============================] - 0s 339us/step - loss: 1.3430 - acc: 0.4383 - val_loss: 1.4476 - val_acc: 0.3625

In [93]:
?model.fit()